# DATA SCIENCE CHALLENGE #3 - SIMILITUD ENTRE PRODUCTOS

+ Un desafío constante en MELI es el de poder agrupar productos similares utilizando algunos atributos de estos como pueden ser el título, la descripción o su imagen;
+ Para este desafío tenemos un dataset `items_titles.csv` que tiene títulos de 30 mil productos de 3 categorías diferentes de Mercado Libre Brasil;
+ El objetivo del desafío es poder generar una Jupyter notebook que determine cuán similares son dos títulos del dataset `item_titles_test.csv`, generando como output un listado de la forma:

|    ITE_ITEM_TITLE   |   ITE_ITEM_TITLE    |  Score Similitud (0,1)  |
|:-------------------:|:-------------------:|:-----------------------:|
|   Zapatillas Nike   |  Zapatillas Adidas  |           0.5           |
|   Zapatillas Nike   |  Zapatillas Nike    |            1            |

donde ordenando por score de similitud podamos encontrar los pares de productos más similares en nuestro dataset de test.

## OVERVIEW OF THE FIRST PROPOSED SOLUTION

This dataset provides a subset of the items likely to be found in production, and none of the datasets provide annotations of which products are known to be similar to other products. Yet, the task is that given any two items with their descriptions, we need to output a score between 0 and 1 indicating how much similar are those two items given their descriptions. From the example above, we can see that if half of the words are shared between items, then their similar should be 0.5. 

There are quite a few ways to tackle this task, but I believe the first attempt should be the simplest one possible - so that we can know whether any more ellaborate approach is bringing further value or not. As such, I decided to implement a brute force algorithm that will create simple vector representations for each item based on its description and, based on the cosine similarity between those vectors, will find the _n_ items whose descriptions are the most similar ones. The end product of this algoritm will be a dataframe containing each pair of items and their similarity score in each row.

An important aspect of the proposed approach is that we will use the training data to build the vocabulary used by the algorithm - taking care of unknown tokens and also assigning a special token to any word containing a digit. Finally, if put to production, this algorithm would work by receiving an input description and looking at its knowledge base for the other items whose descriptions are close to it - as such, it would not work by crossing all items with all other items in its knowledge base.

## Loading modules and functions

In [1]:
import pandas as pd # dataframes
import numpy as np # arrays
import matplotlib.pyplot as plt # plotting utilities
from unidecode import unidecode # special characters
from collections import defaultdict # dictionary
from nltk.tokenize import word_tokenize # sentence tokenization
from collections import Counter # counter dictionary 
import re # regex
import nltk # text
import os # operational system
from tqdm import tqdm # progress bar

## Loading the data

In [2]:
# loading the dataset containing the instances that will be used to develop the algorithm
df_train = pd.read_csv(filepath_or_buffer = '../data/items_titles.csv')
# loading the dataset containing the instances that will be used to test the algorithm
df_test = pd.read_csv(filepath_or_buffer = '../data/items_titles_test.csv')

## Pre-processing the text

Creating a function to pre-process and tokenize each sentence into a list of words.

In [3]:
def pre_process_text(sentence):
    """
    Pre-processes the sentence by lower casing all characters, replacing any special characters by their standard 
    counterparts (e.g., 'ê' -> e) and splitting the sentence based on whitespace.
    
    Arguments
    ---------
    sentence: string
        A string containing the sentence that will be pre-processed.
    
    Returns
    -------
        A list of strings where each element is a word found in the original sentence
    """
    decoded_sentence = unidecode(sentence) # stripping special characters
    decoded_sentence = decoded_sentence.lower() # lower casing the entire sentence
    decoded_sentence = re.sub('[^\w\s\d]', ' ', decoded_sentence) # removing any punctuation
    return word_tokenize(text = decoded_sentence, language = "portuguese") # returning the tokenized sentence

Next, we will tokenize all sentences with the item descriptions from the training dataset, and use those tokens to create the vocabulary that will be used to score the test dataset.

In [4]:
## list of words within each sentence
train_sentences = df_train.ITE_ITEM_TITLE.apply(pre_process_text).tolist()
## getting the list of unique tokens across all sentences
vocabulary = set([word for sentence in train_sentences for word in sentence])

We will create a function to encode the training vobulary, which will give an integer index to each token, a special index to any token containing a digit and will also create a special token and index for unknowns.

In [5]:
def encode_vocabulary(vocabulary, unknown_idx = 0, digit_idx = 1):
    """
    Integer-encode a vocabulary.
    
    Arguments
    ---------
    vocabulary: dict
        Input dictionary containing the tokens found in a corpus.
    digit_idx: int, defaults to 1
        Integer encoding the index that will be given to any token containing a digit

    Returns
    -------
    word_to_idx: dict
        Dictionary used to encode each token to a unique integer.
    idx_to_word: dict
        Dictionary used to encode each integer to a token.
    """
    # creating default dictionaries to store the encoded values
    word_to_idx = defaultdict(int)
    idx_to_word = defaultdict(int)
    
    # initializing the idx_to_word dictionary with the values for the unknown and digit ids
    idx_to_word[unknown_idx] = '__UNK__'
    idx_to_word[digit_idx] = '__DGT__'
    
    #
    counter = unknown_idx + digit_idx + 1
    
    # iterating across the non-digit tokens in the vocabulary to assign an index to them
    for token in vocabulary:
        if token.isdigit() or any(character.isdigit() for character in token):
            pass
        else:
            word_to_idx[token] = counter
            idx_to_word[counter] = token
            counter += 1
    return word_to_idx, idx_to_word

Creating our training vocabulary, containing all words that appear in the train dataset, along with their indexes.

In [6]:
# defining the default indexes that will be used for the unknown and digit tokens
idx_for_unknown = 0
idx_for_digits = 1
# getting the encoded representations of tokens and their indexes
word2idx, idx2word = encode_vocabulary(vocabulary = vocabulary, unknown_idx = idx_for_unknown, digit_idx = idx_for_digits)

## Implementing a baseline approach

Creating a series of functions that will:  

+ Get an input sentence and encode it into the indexes of its tokens - this is based on the tokens seem in the training dataset;
+ Get an input list of token indexes and create a simple vector representation from it;
+ Find the most similar itens to a given target item according to their vector representations.

In [7]:
def encode_sentence(sentence, word_to_idx_dict, unknown_idx = 0, digit_idx = 1):
    """
    Integer-encode an input sentence based on the indexes given to each token in a vocabulary, while also taking
    care to given specific encoding values to unknown words and for any token where a digit is found.
    
    Arguments
    ---------
    sentence: string
        A string containing the input sentence that will be encoded.
    word_to_idx_dict: dict
        Dictionary mapping the tokens to their integer indexes.
    unknown_idx: int, defaults to 0
        Integer encoding the index that will be given to any unknown token
    digit_idx: int, defaults to 1
        Integer encoding the index that will be given to any token containing a digit
    
    Returns
    -------
    sentence_idx: list of int
        List of integers representing the indexes from each of the tokens found in a given sentence
    """
    # pre-processing and tokenizing the sentence
    tokenized_sentence = pre_process_text(sentence)
    # creating and empty list to store the indexes of the sentence tokens
    sentence_idx = []
    # iterating across the tokens in the tokenized sentence to extract their indexes
    for token in tokenized_sentence:
        # assigning any token containing a digit to its own integer index
        if token.isdigit() or any(character.isdigit() for character in token):
            sentence_idx.append(digit_idx)
        # using the get method of the dictionary to extract the word index or, if that does not exist, assigning
        # the unknown word index
        else:
            sentence_idx.append(word2idx.get(token, unknown_idx))
    
    return sentence_idx

In [8]:
def one_hot_array(indexes, vocab_size):
    """
    Creates a one-hot vector representation of an input sentence that have already been encoded.

    Arguments
    ---------
    indexes: list of int
        List of integers representing the indexes from each of the tokens found in a given sentence.
    vocab_size: int
        Size of the original vocabulary.
    
    Returns
    -------
        A numpy array of shape (vocab_size,) containing the vector representation for a sentence represented by
        a sequence of tokens.
    """
    # creating a matrix of zeroes to accumulate the one hot encoded representation of each token in the sentence
    # independently - each row is a token, each column a token index
    one_hot = np.zeros((len(indexes), vocab_size))
    # populating the zeroes matrix according to the number of times each token appears in the input sentence
    for row, idx in enumerate(indexes):
        one_hot[row][idx] += 1
    # creating a vector representation of the input sentence based on the column-wise average of the one hot encoding
    # this array represents the average content of the sentence
    return np.mean(one_hot, axis = 0)

In [9]:
def score_similitud(target_sentence, possible_sentences, sentence_encoder, word_to_idx_dict, unknown_idx, digit_idx, ohe_function, n_closest = 5):
    """
    Extract the itens with the most similar descriptions given a target item, according to the cosine similarity between their
    vector representations.
    
    Arguments
    ---------
    target_sentence: string
        A string containing the target sentence. It represents the item we are interested in finding other similar to it.
    possible_sentences: list of strings
        List containing the sentences that represent the itens that will be used to search for those that most closely 
        matches with the sentence in target_sentence.
    sentence_encoder: python function
        Python function used to integer-encode the target sentence.
    word_to_idx_dict: dict
        Dictionary mapping the tokens to their integer indexes.
    unknown_idx: int, defaults to 0
        Integer encoding the index that will be given to any unknown token.
    digit_idx: int, defaults to 1
        Integer encoding the index that will be given to any token containing a digit.
    ohe_function: python function
        Python function used to one hot encode a list of token indexes.
    n_closest: int, defaults to 5
        Integer specifying the number of similar itens that will be returned by the function.
    
    Returns
    -------
    sentence_similarity: list of tuples
        Each tuple contains the item name and its cosine similarity with the target item used as an argument in target_sentence.
        This length of this list is equal to the number of itens similar to the target item, which can be tuned by the function
        argument `n_closest`; in addition, the tuples are ordered from that with the largest cosine similarity to the one with
        the lowest, such that the first item in the list is the one is expected to be the most similar with the target item.
    """
    # extracting the vocabulary size from the word to index dictionary - this is the maximum index value in this dictionary
    max_token_idx = max(word_to_idx_dict.values())
    # encoding the target sentence
    target_tokens = sentence_encoder(target_sentence, word_to_idx_dict, unknown_idx, digit_idx)
    # one hot encoding the target sentence
    ohe_target = ohe_function(indexes = target_tokens, vocab_size = max_token_idx + 1)
    
    # creating an empty list to store the tuple containing the cosine similarity between the target setence and
    # every other sentence in possible_sentences
    sentence_similarity = []
    
    # iterating across each of the other possible sentences
    for proposal in possible_sentences:
        # encoding the proposed sentence
        proposal_tokens = sentence_encoder(proposal, word_to_idx_dict, unknown_idx, digit_idx)
        # one hot encoding the proposed sentence
        ohe_proposal = ohe_function(indexes = proposal_tokens, vocab_size = max_token_idx + 1)
        
        # calculating the cosine similarity between the vector representation of the target sentence and the 
        # proposed sentence
        numerator = np.dot(ohe_target[1:], ohe_proposal[1:])
        denominator = np.linalg.norm(ohe_target[1:]) * np.linalg.norm(ohe_proposal[1:])
        cosine = numerator / denominator if not np.isclose(0.0, denominator) else 0.0
        
        # appending the tuple containing the proposed sentence and the cosine similarity between that sentence
        # and the target sentence into the list of sentence similarities
        sentence_similarity.append((proposal, cosine))
    
    # reordering the list of sentence similarity so that tuples with higher cosine similarity comes first, and
    # extracting the n_closest matches to the target sentence according to this metric
    sentence_similarity = sorted(sentence_similarity, key = lambda x: x[-1], reverse = True)[:n_closest]
    
    return sentence_similarity

Finding each of the itens that are most similar to each other.

In [10]:
## creating an empty dataframe to store the results of the brute force item similarity search
sample_output = pd.DataFrame()

## iterating across each of the itens available in the test dataset
for item in tqdm(df_test.iloc[:, 0].tolist()[:5]):
    # extracting the five most similar itens to each other item
    item_scoring = score_similitud(
        item, possible_sentences = df_train.iloc[:, 0].tolist(), 
        sentence_encoder = encode_sentence, word_to_idx_dict = word2idx, 
        unknown_idx = idx_for_unknown, digit_idx = idx_for_digits, 
        ohe_function = one_hot_array, n_closest = 5
    )
    # tidying up the results to keep track of the item pairs
    item_output = pd.DataFrame(item_scoring, columns = ['Similar Items', 'Score Similitud'])
    item_output['Target Item'] = item
    item_output = item_output[['Target Item', 'Similar Items', 'Score Similitud']]
    # concatenating the output of the current iteration with the one from the previous iterations
    sample_output = pd.concat([sample_output, item_output])

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:38<00:00,  7.73s/it]


In [11]:
## dropping any itens have the same item description - otherwise we woudn't need this algorithm
sample_output = sample_output.loc[sample_output['Target Item'] != sample_output['Similar Items']]

## taking a look at the results
sample_output

,Target Item,Similar Items,Score Similitud
0,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Infantil Olympikus Valente Kids Masculino,0.833333
1,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Esporte Masculino,0.707107
2,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Kids Masculino Esportivo Olympikus V-zero,0.617213
3,Tênis Olympikus Esporte Valente - Masculino Kids,Tenis Infantil Masculino Olympikus Spider Kids...,0.617213
4,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Esporte Olympikus Veloz,0.612372
0,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...,Bicicleta Barra Forte Aro 24,0.516398
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...,Bicicleta Samy Cross Aro 20 C/ Aros Aero,0.510310
2,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...,Bicicleta Sport Aro 29 C/ Susp. 21v C/shim Tam...,0.503953
3,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...,Bicicleta Sport Aro 29 C/ Susp. 21 V C/shim Ta...,0.492366
4,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...,Bicicleta 29 Elleven Rocker 24v Hidráulico C/ ...,0.456435


In [12]:
## saving the results locally
if not os.path.exists(path = '../outputs/'):
    os.mkdir(path = '../outputs/')
sample_output.to_csv(path_or_buf = '../outputs/sample_output_challenge_3.csv', index = False)

## Where to go from here?

My idea is that this very simple algorithm could be used to measure at least to things: (1) classification accuracy and (2) computation time. I would then take their values as baselines to this problem, and try to improve on them attempting any of the following (and/or their combinations):

+ Pre-calculate the vector representation of the items in the training data and use them in the `score_similitud` function above instead of having to do it all from scratch. The main advantage of this feature would be to speed up the inference time for the function;  
+ Pre-calculate not only the vector representations from the training data but also their L2 norm for use in the denominator for the cosine similiarity. This feature should also speed up the inference time;
+ Replace the `for` loop for a vectorized version of the operation - _e.g._, use a dot product between the target item vector representation in matrix format and the vector represetation of the other items. This should speed up the inference time considerably.
+ Use pre-trained word vectors instead of the simple average of the one hot encoded representation of each word;  
+ Use a locally sensitive hashing or some technique alike to partition the itens into smallers groups, and then try to conduct item similarity search within each of these hash buckets;   
+ Manually annotate a few instances from the training dataset and use it to train a supervised algorithm (_e.g._, a siamese network);  
+ Attempt to automate the data annotation process to train a supervised algorithm by:  
    * Randomly selecting an item and:
        * Creating a copy of it that would be lacking from 0 to k of its words. This copy and the original item would be taken as instances representing similar itens;  
        * Randomly picking another item and dropping from 0 to k of its words. This copy and the original item would be taken as instances representing dissimilar itens.
    * Then, I would create some new training data from the item pairs created on the above step; and,
    * I would them train a supervised algorithm on top of this data.